In [1]:
import pandas as pd
import numpy as np

In [ ]:
# genes-pathways annotation

path = './kegg_pathway/kegg_hsa.gmt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_genes_dict = {}
for i in files:  
    paways_genes_dict[i.split('\t')[0].split('_')[0]] = i.replace('\n','').split('\t')[2:] 

print(paways_genes_dict)

In [ ]:
#mirna-pathways annotation
path = './kegg_pathway/kegg_anano.txt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_mirna_dict = {}
for i in files:
     keys = i.split(',')[0].split('|')[1]
     values1 = i.split(',')[1:-1]
     values2 =  i.split(',')[-1].replace('\n','')
     values1.append(values2)
     values1 =list(set(values1)) 
     paways_mirna_dict[keys] = values1

print(paways_mirna_dict)

In [ ]:
# Compute the intersection of paways_genes_dict and paways_mirna_dict passes
union_kegg = list(set(paways_genes_dict.keys()).intersection(set(paways_mirna_dict.keys())))
print(union_kegg)

In [ ]:
paways_genes_dicts ={}
paways_mirna_dicts ={}
# Obtain the common pathway information of genes and miRNAs
for i in union_kegg:
    paways_genes_dicts[i] = paways_genes_dict[i]
    
for i in union_kegg:
    paways_mirna_dicts[i] = paways_mirna_dict[i]    
print(paways_genes_dicts)
print(len(paways_genes_dicts))
print(len(paways_mirna_dicts))

In [6]:

genes_existed_pathway = []
mirna_existed_pathway = []

# Merge all the genes in the public pathways in the dictionary paways_genes_dicts
# and store the final result in a de-weighted set genes_existed_pathway.
for index in paways_genes_dicts.keys():
    genes_existed_pathway = genes_existed_pathway+ list(paways_genes_dicts[index])
genes_existed_pathway = set(genes_existed_pathway)


for index in paways_mirna_dicts.keys():
    mirna_existed_pathway = mirna_existed_pathway+ list(paways_mirna_dicts[index])
mirna_existed_pathway = set(mirna_existed_pathway)


In [ ]:
print(genes_existed_pathway)
print(len(genes_existed_pathway))
print(len(mirna_existed_pathway))

In [8]:
#Loading data
snv_data = pd.read_csv("./lihc_data/snv_data.csv",index_col = 0)

miRNA_data = pd.read_csv("./lihc_data/miRNA_data.csv",index_col = 0)

mRNA_data = pd.read_csv("./lihc_data/mRNA_data.csv",index_col = 0)

example_case = pd.read_csv('./lihc_data/response.csv',index_col=0)

In [ ]:
print(snv_data.shape)
print(mRNA_data.shape)
print(miRNA_data.shape)
print(example_case.shape)


In [10]:
##Put the Omics data into lists separately
union_gene_snv = list(snv_data.columns)
union_gene_miRNA = list(miRNA_data.columns)
union_gene_mRNA = list(mRNA_data.columns)

In [ ]:
pathway_union = list(paways_genes_dicts.keys())
len(pathway_union)

In [12]:
# Combine union_gene_snv and union_gene_mRNA into a two-dimensional array
mask_list = [union_gene_snv,union_gene_mRNA]
# Stored are correspondences between pathways and snv, mRNAs and miRNAs.
gene_pathway_bp_dfs = []  


for i in range(len(mask_list)):
    # Creating a 238 * 1000 matrix of 0 values
    pathways_genes = np.zeros((len(pathway_union), len(mask_list[i]))) 
    for p  in pathway_union:
        # Get a list of genes in the pathway
        gs = paways_genes_dicts[p]
        # Find the gene index in gs that exists in mask_list[i]
        g_inds = [mask_list[i].index(g) for g in gs if g in mask_list[i]]
        # Get the index of the current route in pathway_union
        p_ind = pathway_union.index(p)
        # Setting the elements of the matrix corresponding to pathways and genes to 1 indicates the association between them.
        pathways_genes[p_ind, g_inds] = 1
    # Use the pathways_genes matrix to create a DataFrame whose indexes are pathway names and columns are lists of genes.
    gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=mask_list[i])
    
   # gene_pathway_bp = gene_pathway_bp.loc[:, (gene_pathway_bp != 0).any(axis=0)]
    gene_pathway_bp_dfs.append(gene_pathway_bp)
    
# The same method gets a DataFrame whose index is the pathway name and columns are the miRNA list.
pathways_genes = np.zeros((len(pathway_union), len(union_gene_miRNA))) 
for p  in pathway_union:
    gs = paways_mirna_dicts[p]
    g_inds = [union_gene_miRNA.index(g) for g in gs if g in union_gene_miRNA]
    p_ind = pathway_union.index(p)
    pathways_genes[p_ind, g_inds] = 1
gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=union_gene_miRNA)


# gene_pathway_bp = gene_pathway_bp.loc[:, (gene_pathway_bp != 0).any(axis=0)]
gene_pathway_bp_dfs.append(gene_pathway_bp)
    
# gene_pathway_bp_dfs.to_cvs("gene_pathway_bp_dfs.csv")
combined_df = pd.concat(gene_pathway_bp_dfs, ignore_index=True)

# Write the merged DataFrame to a CSV file
#combined_df.to_csv("gene_pathway_bp_dfs.csv", index=False)

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform, Initializer
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D,Layer
from tensorflow.keras import initializers,activations,regularizers
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2

In [14]:
#Biological Pathway Module
# A matrix of relationships between mRNAs, miRNAs, SNVs and their respective pathways was constructed with multi-omics data
class Biological_module(Layer):
    #initialization Parameter 
    def __init__(self, units, mapp=None, nonzero_ind=None, kernel_initializer='glorot_uniform', W_regularizer=None,
                 activation='tanh', use_bias=True,bias_initializer='zeros', bias_regularizer=None,
                 bias_constraint=None,**kwargs):
        self.units = units
        self.activation = activation
        self.mapp = mapp
        self.nonzero_ind = nonzero_ind
        self.use_bias = use_bias
        
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(W_regularizer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activation_fn = activations.get(activation)
        super(Biological_module, self).__init__(**kwargs)

     #build module
    def build(self, input_shape):
        input_dim = input_shape[1]
        if not self.mapp is None:
            self.mapp = self.mapp.astype(np.float32)
        if self.nonzero_ind is None:
            nonzero_ind = np.array(np.nonzero(self.mapp)).T
            self.nonzero_ind = nonzero_ind
        self.kernel_shape = (input_dim, self.units) 
        nonzero_count = self.nonzero_ind.shape[0]  
         # node and node  connection nunber 
        self.kernel_vector = self.add_weight(name='kernel_vector',
                                             shape=(nonzero_count,),
                                             initializer=self.kernel_initializer,
                                             regularizer=self.kernel_regularizer,
                                             trainable=True)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer
                                        )
        else:
            self.bias = None
        super(Biological_module, self).build(input_shape)  
     #Define a customized neural network layer with sparse connections, optional bias terms and activation functions
    def call(self, inputs):
              
        trans = tf.scatter_nd(tf.constant(self.nonzero_ind, tf.int32), self.kernel_vector,
                           tf.constant(list(self.kernel_shape)))
        output = K.dot(inputs, trans)
    
        if self.use_bias:
            output = K.bias_add(output, self.bias)
            
        if self.activation_fn is not None:
            output = self.activation_fn(output)

        return output
    # get module config
    def get_config(self):
        config = {
            'units': self.units,
            'activation': self.activation,
            'use_bias': self.use_bias,
            'nonzero_ind': np.array(self.nonzero_ind),
          
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),

            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'W_regularizer': regularizers.serialize(self.kernel_regularizer),

        }
        base_config = super(Biological_module, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
      
        return (input_shape[0], self.units)

In [15]:
#Multiple Attention Module,Learning inter-sample correlations through multiple attention mechanisms
class MultiHeadAttention(Layer):
    def __init__(self, output_dim, num_heads,dropout_rate=0.1, W_regularizer=None, **kwargs):
        self.output_dim = output_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.kernel_regularizer = regularizers.get(W_regularizer)
        super(MultiHeadAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert self.output_dim % self.num_heads == 0, "Output dimension must be divisible by the number of heads."
        self.depth = self.output_dim // self.num_heads

        self.query_kernel = self.add_weight(name='query_kernel',
                                            shape=(input_shape[-1], self.output_dim),
                                            initializer='uniform',
                                            regularizer=self.kernel_regularizer,
                                            trainable=True)
        self.key_kernel = self.add_weight(name='key_kernel',
                                          shape=(input_shape[-1], self.output_dim),
                                          initializer='uniform',
                                          regularizer=self.kernel_regularizer,
                                          trainable=True)
        self.value_kernel = self.add_weight(name='value_kernel',
                                            shape=(input_shape[-1], self.output_dim),
                                            initializer='uniform',
                                            regularizer=self.kernel_regularizer,
                                            trainable=True)
        self.dropout = Dropout(self.dropout_rate)
        super(MultiHeadAttention, self).build(input_shape)

    def call(self, x,training=False):
        batch_size = tf.shape(x)[0]

        # Linear projections
        Q = tf.tensordot(x, self.query_kernel, axes=[-1, 0])  # (batch_size, seq_len, output_dim)
        K = tf.tensordot(x, self.key_kernel, axes=[-1, 0])    # (batch_size, seq_len, output_dim)
        V = tf.tensordot(x, self.value_kernel, axes=[-1, 0])  # (batch_size, seq_len, output_dim)

        # Reshape to (batch_size, seq_len, num_heads, depth)
        Q = tf.reshape(Q, (batch_size, -1, self.num_heads, self.depth))
        K = tf.reshape(K, (batch_size, -1, self.num_heads, self.depth))
        V = tf.reshape(V, (batch_size, -1, self.num_heads, self.depth))

        # Transpose to (batch_size, num_heads, seq_len, depth)
        Q = tf.transpose(Q, perm=[0, 2, 1, 3])
        K = tf.transpose(K, perm=[0, 2, 1, 3])
        V = tf.transpose(V, perm=[0, 2, 1, 3])

        # Scaled dot-product attention
        QK = tf.matmul(Q, K, transpose_b=True)  # (batch_size, num_heads, seq_len, seq_len)
        QK = QK / tf.math.sqrt(tf.cast(self.depth, tf.float32))
        QK = tf.nn.softmax(QK, axis=-1)
        QK = self.dropout(QK, training=training)

        # Weighted sum of values
        attention_output = tf.matmul(QK, V)  # (batch_size, num_heads, seq_len, depth)

        # Transpose and reshape back to (batch_size, seq_len, output_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        attention_output = tf.reshape(attention_output, (batch_size, -1, self.output_dim))
        attention_output = self.dropout(attention_output, training=training)

        return attention_output

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'num_heads': self.num_heads,
            'dropout_rate': self.dropout_rate,
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
        }
        base_config = super(MultiHeadAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.output_dim)


In [16]:
#build a model
def create_model(snv_data,mRNA_data,miRNA_data): 

    #Input Layer Definition
    S_inputs_snv = Input(shape=(snv_data.shape[1],), dtype='float32',name= 'snv_inputs')
    S_inputs_mRNA = Input(shape=(mRNA_data.shape[1],), dtype='float32',name= 'mRNA_inputs')
    S_inputs_miRNA = Input(shape=(miRNA_data.shape[1],), dtype='float32',name= 'miRNA_inputs')
    
    #Calling the Biological Pathway Module
    h0_snv = Biological_module(gene_pathway_bp_dfs[0].shape[0],mapp =gene_pathway_bp_dfs[0].values.T, name = 'h0_snv',W_regularizer=l2(0.001))(S_inputs_snv)
    h0_mRNA = Biological_module(gene_pathway_bp_dfs[1].shape[0],mapp =gene_pathway_bp_dfs[1].values.T, name = 'h0_mRNA',W_regularizer=l2(0.001))(S_inputs_mRNA)
    h0_miRNA = Biological_module(gene_pathway_bp_dfs[2].shape[0],mapp =gene_pathway_bp_dfs[2].values.T, name = 'h0_miRNA',W_regularizer=l2(0.001))(S_inputs_miRNA)

    #Multiple Self-Attention Mechanism
    atten1 = MultiHeadAttention(output_dim=128, num_heads=16, W_regularizer=l2(0.001))(h0_snv)
    atten2 = MultiHeadAttention(output_dim=128, num_heads=16, W_regularizer=l2(0.01))(h0_mRNA)
    atten3 = MultiHeadAttention(output_dim=128, num_heads=16, W_regularizer=l2(0.01))(h0_miRNA)

    #Hierarchical feature fusion
    atten12 = tf.keras.layers.concatenate([atten1, atten2])
    atten13 = tf.keras.layers.concatenate([atten1, atten3])
    feature_tal = tf.keras.layers.concatenate([atten12, atten13])
    #full connectivity layer
    h2 = tf.keras.layers.Dense(64, activation='tanh')(feature_tal)
    h3 = tf.keras.layers.Dense(32, activation='tanh')(h2)
    h4 = tf.keras.layers.Dense(8, activation='tanh')(h3)
    h5 = tf.keras.layers.Dense(1, activation='sigmoid')(h4)

    # Convert the output from a 3D array to a 2D array (model modification)
    # Use tf.squeeze to remove dimensions of length 1
    h5 = tf.squeeze(h5, axis=-2)

    model = Model(inputs=[S_inputs_snv,S_inputs_mRNA,S_inputs_miRNA], outputs=h5)

    model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,decay=0.0001) 
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [17]:
##Evaluation function
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score   
from sklearn.metrics import precision_recall_curve


def get_metrics(true_score,pre_score,pre_probe):
    
    # Calculate the True Positive Rate (TPR) and False Positive Rate (FPR) and the corresponding thresholds. pos_label=1 specifies the label of the positive class

    fpr, tpr, thresholds = metrics.roc_curve(true_score, pre_probe, pos_label=1)

    # Calculate Area Under the Curve (AUC) 
    auc = metrics.auc(fpr, tpr)
    
    # Average Precision Score, AUPR
    aupr = average_precision_score(true_score, pre_probe)
    
    # Precision and Recall
    pre, rec, thresholds = precision_recall_curve(true_score, pre_probe) 

    # AUPRC
    auprc  = metrics.auc(rec, pre)
    
    # Accuracy
    accuracy = accuracy_score(true_score,pre_score)
    
    # F1 Score
    f1 = metrics.f1_score(true_score, pre_score)
    
    # Calculation of the precision rate
    precision = metrics.precision_score(true_score,pre_score)
    # Calculating Recall
    recall = metrics.recall_score(true_score,pre_score)

    return precision,accuracy,recall,f1,auc,aupr,auprc



#Calculation of indicators for each assessment
def evaluates(y_test, y_pred):
    
    auc = metrics.roc_auc_score(y_test,y_pred)
    
    aupr = average_precision_score(y_test, y_pred)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)    
    auprc  = metrics.auc(recall, precision)
    
    pp = [1 if index>=0.5  else 0 for index in  y_pred ]
    
    pre = metrics.precision_score(y_test,pp)
    
    f1 = metrics.f1_score(y_test,pp)
    
    rec = metrics.recall_score(y_test,pp)
    
    acc = metrics.accuracy_score(y_test,pp)
    
    print(confusion_matrix(y_test,pp))
    return pre,acc,rec,f1,auc,aupr,auprc

In [18]:
snv_data = snv_data.loc[example_case.index]

miRNA_data = miRNA_data.loc[example_case.index]

mRNA_data = mRNA_data.loc[example_case.index]

example_case = example_case.loc[example_case.index]

y = example_case['response'].values


In [ ]:

n_samples =example_case['response'].values

print(len(n_samples),n_samples.sum(),(len(n_samples) -n_samples.sum()))

x_0 =  len(n_samples) / (2*  (len(n_samples) -n_samples.sum()))
# Sample size for each event as a proportion of the total sample size
x_1 =  len(n_samples) / (2*  n_samples.sum())

print(x_0,x_1)

In [ ]:
#  5-fold cross validation
from sklearn.model_selection import StratifiedKFold
# n_splits=5: This parameter specifies the number of folds for cross-validation. In this example, the dataset will be divided into 5 parts.
# shuffle=True: This parameter specifies whether the data should be randomly disrupted before splitting.
# Setting it to True means that the data will be randomized before the fold is created, which helps to reduce possible biases in the training order.
# random_state=1029: This parameter is used to set the seed of the random number generator to ensure that you get the same random permutation every time you run the code.
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1029) 

kfscore = []
i = 0
for train_index, test_index in skf.split(snv_data.values,y):
    
    snv_train_x = snv_data.values[train_index]
    snv_test_x  = snv_data.values[test_index]

    mRNA_train_x = mRNA_data.values[train_index]
    mRNA_test_x  = mRNA_data.values[test_index]

    miRNA_train_x = miRNA_data.values[train_index]
    miRNA_test_x  = miRNA_data.values[test_index]

    train_y  = y[train_index]
    test_y   = y[test_index]

    model = create_model(snv_data,mRNA_data,miRNA_data)
    model.fit( {"snv_inputs": snv_train_x,  "mRNA_inputs": mRNA_train_x, 'miRNA_inputs':miRNA_train_x},train_y,
                 validation_data=({"snv_inputs": snv_test_x, "mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x},test_y),
                 epochs=200,batch_size = 64,class_weight = {0:x_0,1:x_1})  

    y_pred = model.predict({"snv_inputs": snv_test_x, "mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x})

    y_score = [1 if index>=0.5  else 0 for index in  y_pred]

    evaluate_epoch = get_metrics(test_y,y_score,y_pred)
    print(evaluate_epoch)
    kfscore.append(evaluate_epoch)
    
results = list(np.array(kfscore).sum(axis= 0)/5.0)
print('Cross validated results :  ACC = {}, REC = {}, F1 = {}, AUC = {}, AUPR ={}'.format(results[1],results[2],results[3],results[4],results[5]))

In [21]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
train_index, test_index =  train_test_split(snv_data.values, test_size=0.2, random_state=0)
# print(train_index)
# print(test_index)
X_train_snv, X_test_snv, X_train_mRNA, X_test_mRNA, X_train_miRNA, X_test_miRNA, y_train, y_test = train_test_split(
    snv_data.values, mRNA_data.values, miRNA_data.values, y, test_size=0.2, random_state=42
)

In [ ]:
#Interpretability of the model
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Creating Data Sets
X_train_snv, X_test_snv, X_train_mRNA, X_test_mRNA, X_train_miRNA, X_test_miRNA, y_train, y_test = train_test_split(
    snv_data.values, mRNA_data.values, miRNA_data.values, y, test_size=0.2, random_state=42
)

# Creating Models
model = create_model(snv_data,mRNA_data,miRNA_data)
#model training
model.fit( {"snv_inputs": X_train_snv,  "mRNA_inputs": X_train_mRNA, 'miRNA_inputs':X_train_miRNA},y_train,
                validation_data=({"snv_inputs": X_test_snv, "mRNA_inputs": X_test_mRNA,'miRNA_inputs':X_test_miRNA},y_test),
                epochs=200,batch_size = 64,class_weight = {0:x_0,1:x_1})

# Predicting or evaluating in this model
y_pred = model.predict({"snv_inputs": X_test_snv, "mRNA_inputs": X_test_mRNA,'miRNA_inputs':X_test_miRNA})

#Training data, splicing training data from three omics
X_train = [X_train_snv[:100], X_train_mRNA[:100], X_train_miRNA[:100]]

# Creating the SHAP interpreter
explainer = shap.DeepExplainer(model, X_train)


shap.explainers._deep.deep_tf.op_handlers["AddN"] = shap.explainers._deep.deep_tf.passthrough

shap.explainers._deep.deep_tf.op_handlers["BatchMatMulV2"] = shap.explainers._deep.deep_tf.passthrough
X_test  = [X_test_snv, X_test_mRNA, X_test_miRNA]


# Calculating the SHAP value
shap_values = explainer.shap_values(X_test )


# Visualize the SHAP value
feature_snv = snv_data.columns.to_numpy()  # First input feature name
feature_mRNA = mRNA_data.columns.to_numpy()  # Feature name for the second input
feature_miRNA = miRNA_data.columns.to_numpy()  # Third input feature name
feature_names = [feature_snv, feature_mRNA, feature_miRNA]

# summary Plot
shap.summary_plot(shap_values[0][0], X_test[0], feature_names=feature_snv, plot_type="bar",max_display=10)
shap.summary_plot(shap_values[0][1], X_test[1], feature_names=feature_mRNA, plot_type="bar",max_display=10)
shap.summary_plot(shap_values[0][2], X_test[2], feature_names=feature_miRNA, plot_type="bar",max_display=10)

for i, shap_values_array in enumerate(shap_values[0]):
     shap.summary_plot(shap_values_array, X_test[i], feature_names=feature_names[i], plot_type="bar")

# summary Plot
shap.summary_plot(shap_values[0][0], X_test[0], feature_names=feature_snv,max_display=10)
shap.summary_plot(shap_values[0][1], X_test[1], feature_names=feature_mRNA, max_display=10)
shap.summary_plot(shap_values[0][2], X_test[2], feature_names=feature_miRNA, max_display=10)


#  Beeswarm 
shap_values_snv = shap_values[0][0]  # HAP value of snv
shap_values_mRNA = shap_values[0][1]  # HAP value of mRNA
shap_values_miRNA = shap_values[0][2]  # HAP value of miRNA
# snv Beeswarm
shap.plots.beeswarm(shap.Explanation(shap_values_snv, feature_names=feature_snv))
# mRNA Beeswarm
shap.plots.beeswarm(shap.Explanation(shap_values_mRNA, feature_names=feature_mRNA))
# miRNA Beeswarm
shap.plots.beeswarm(shap.Explanation(shap_values_miRNA, feature_names=feature_miRNA))

# Heatmap

fig, ax = plt.subplots()
# snv Heatmap
shap.plots.heatmap(shap.Explanation(shap_values_snv, feature_names=feature_snv))
# mRNA Heatmap
shap.plots.heatmap(shap.Explanation(shap_values_mRNA, feature_names=feature_mRNA))
# miRNA Heatmap
shap.plots.heatmap(shap.Explanation(shap_values_miRNA, feature_names=feature_miRNA))
# customize the axis object
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)

#If SHAP values are needed for splicing multi-omics features
# Splicing SHAP values from different omics
shap_values_combined = np.concatenate([shap_values_snv, shap_values_mRNA, shap_values_miRNA], axis=1)
# Splice feature name
feature_names_combined = np.concatenate([feature_snv, feature_mRNA, feature_miRNA])
# Mapping the merged Beeswarm
shap.plots.beeswarm(shap.Explanation(shap_values_combined, feature_names=feature_names_combined))
# Plotting the merged Heatmap
shap.plots.heatmap(shap.Explanation(shap_values_combined, feature_names=feature_names_combined))

